Code supporting the article. 
As the structures of the compounds are not provided, only the morphological profile based classifier can be tested. 

In [2]:
import pandas as pd
import numpy as np

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Draw, MACCSkeys, rdFMCS, MolStandardize
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.ML.Cluster import Butina

from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import recall_score, balanced_accuracy_score, matthews_corrcoef, accuracy_score, precision_score
from sklearn.neighbors import KNeighborsClassifier

# getting acute toxicity label

In [5]:
# we load a csv containging the concensus profiles, the acute toxicity label (Tox60), and the butina clusters
cellPainting_consensusMorphologicalProfile_acuteToxicity = pd.read_csv('data/cellPainting_consensusMorphologicalProfile_acuteToxicity.csv')
# we also load the set of selected features
CaravaggioDalitFeatSeat01 = pd.read_csv('data/CaravaggioDalitFeatSeat01.csv').iloc[:, 0].values

# chemical structure based classifier

In [ ]:
# a dataframe containing the smiles
smiles_df

## cleaning smiles

In [ ]:
def standardize(smiles):
    """Standardized the smiles.

    Parameters
    ----------
    smiles : string
        the smiles.
  
    Returns
    -------
    string
        standardized smiles.
    """
    mol = Chem.MolFromSmiles(smiles)
    # removeHs, disconnect metal atoms, normalize the molecule, reionize the molecule
    clean_mol = rdMolStandardize.Cleanup(mol) 
    # if many fragments, get the "parent" (the actual mol we are interested in) 
    parent_clean_mol = rdMolStandardize.FragmentParent(clean_mol)
    # try to neutralize molecule
    uncharger = rdMolStandardize.Uncharger() # annoying, but necessary as no convenience method exists
    uncharged_parent_clean_mol = uncharger.uncharge(parent_clean_mol)    
    te = rdMolStandardize.TautomerEnumerator() # idem
    taut_uncharged_parent_clean_mol = te.Canonicalize(uncharged_parent_clean_mol)
    return Chem.MolToSmiles(taut_uncharged_parent_clean_mol, isomericSmiles=False)

# we call the function to standardized the smiles
smiles_df.loc[:, 'standardized_smiles'] = smiles_df.smiles.apply(standardize)

## morgan fingerprint

In [ ]:
def createChemicalSpace (smiles_df, smilesColumn):
    """Create Morgan fingerprint features.

    Parameters
    ----------
    smiles_df : pandas dataframe
        a dataframe containting a column with smiles.
    smilesColumn : string
        the name of the column containting the smiles.
  
    Returns
    -------
    pandas dataframe
        original pandas dataframe with new columns for the Morgan fp features named '0', '1'....
    """
    #we create a df of unique smiles. This df will be used to return the coordinates
    res_df = smiles_df.loc[:, smilesColumn].drop_duplicates()
    #we get rid of the Nan
    res_df = res_df[res_df.notna()]
    print('Number of unique compounds in data:', len(res_df.values))

    # we transform the smiles into an rdkit object
    rdkitObject = [Chem.MolFromSmiles(m) for m in res_df.values]

    # we create a list of boolean telling if an object has been created
    validRdkitObject = [False if o is None else True for o in rdkitObject]
    
    # we display the smiles that couldn't be converted into a rdkit object
    print ("Error with those smiles: ")
    print(res_df[~np.array(validRdkitObject)])
        
    # we get rid of the lines where the rdkitObject is None in the dataframe containing the smiles
    res_df = res_df[validRdkitObject]
    
    # now we create the list of valid molecules, meaning the list of rdkitObject that are not None
    mols = [o for o in rdkitObject if o is not None ]
    #calculate Morgan fingerprints as bit vectors:
    fps = [AllChem.GetMorganFingerprintAsBitVect(m,2,1024) for m in mols]
    #fpgen = [AllChem.GetRDKitFPGenerator(m), for m in mols]
    
    #now we get the list of bit vectors
    fps_bits = [(np.frombuffer(fp.ToBitString().encode(), 'u1') - ord('0')).tolist() for fp in fps]
    fps_bits = pd.DataFrame(fps_bits)
    fps_bits = fps_bits.astype(bool)

    print('Number of molecules OK in data:', len(mols))
    print('Number of Fingerprints in data:', len(fps_bits))
    
    res_df = pd.concat([res_df.reset_index(drop=True), fps_bits.reset_index(drop=True)], axis=1, sort=False)
    return  pd.merge(smiles_df, res_df, on=smilesColumn, how='left')

smiles_df = createChemicalSpace(smiles_df, 'standardized_smiles' )


## Butina clusters


In [ ]:
def computeButinaClusters(smiles_list, cutoff = 0.7)
    """Create Morgan fingerprint features.

    Parameters
    ----------
    smiles_list : list
        a list of smiles.
    cutoff : float
        the Butina algorithm cutoff value.
  
    Returns
    -------
    pandas dataframe
       a pandas dataframe with the smiles and the butina cluster number
    """
    mols = []
    for smiles in all_smiles:
        mols.append(Chem.MolFromSmiles(smiles))
    fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in mols]
    # calcaulate scaffold sets
    # first generate the distance matrix:
    dists = []
    nfps = len(fps)
    for i in range(1, nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i], fps[:i])
        dists.extend([1 - x for x in sims])
    scaffold_sets = Butina.ClusterData(dists,
                                       nfps,
                                       cutoff,
                                       isDistData=True,
                                      reordering = True)
    butinaCluster = pd.DataFrame( index = all_smiles.index, columns = ['cluster']) 
    for cl in range(len(scaffold_sets)):
        for i in scaffold_sets[cl]:
            butinaCluster.iloc[i]['cluster'] = cl
    return butinaCluster

computeButinaClusters(smiles_df.standardized_smiles)

## classifiers

In [7]:
def veryAcuteClassifier (x, y, model, nb_cross_cross_validation = 10, nb_fold = 10, SEED = 24, case_chemistry = 'known', clusters = None):
    """Run classification models and display average metrics.

    Parameters
    ----------
    x : array-like
        training data.
    y : array-like
        target values.
    model : scikit learn model
        classifier model (not fitted).
    nb_cross_cross_validation : int
        number of X fold cross validation
    nb_fold : int
        number of fold for the cross validation
    SEED : int
        random seed to use for the cross validation
    case_chemistry : string
        'known' for knowwn chemistry case
        'new' for new chemistry case
    clusters :  array-like
        list of clusters for the new chemistry case
  
    Returns
    -------
    NONE
    """

    results_df = pd.DataFrame([])

    for CrossValIteration in range(nb_cross_cross_validation):

        if case_chemistry == 'known':
            skf = StratifiedKFold(n_splits=nb_fold, shuffle = True, random_state = SEED + (CrossValIteration*1))

            for train, test in skf.split(y, y):
                xTrain = x.iloc[train, :]
                xTest = x.iloc[test, :]
                yTrain = y.iloc[train]
                yTest = y.iloc[test]

                model.fit(xTrain, yTrain)

                yPredicted = model.predict(xTest)

                # metric
                BA = balanced_accuracy_score(yTest, yPredicted)
                MCC = matthews_corrcoef(yTest, yPredicted)
                SN = recall_score(yTest, yPredicted)
                SP = 2*BA - SN
                PR = precision_score(yTest, yPredicted)

                results_df = pd.concat( [results_df, pd.DataFrame([[BA, MCC, SN, SP, PR]],
                    columns = ['BA', 'MCC', 'SN', 'SP', 'PR'])])

        elif case_chemistry == 'new':
            sgkf = StratifiedGroupKFold(n_splits=nb_fold, shuffle = True, random_state = SEED + (CrossValIteration*1))

            for train, test in sgkf.split(x, y, clusters):
                xTrain = x.iloc[train, :]
                xTest = x.iloc[test, :]
                yTrain = y.iloc[train]
                yTest = y.iloc[test]

                model.fit(xTrain, yTrain)

                yPredicted = model.predict(xTest)

                # metric
                BA = balanced_accuracy_score(yTest, yPredicted)
                MCC = matthews_corrcoef(yTest, yPredicted)
                SN = recall_score(yTest, yPredicted)
                SP = 2*BA - SN
                PR = precision_score(yTest, yPredicted)

                results_df = pd.concat( [results_df, pd.DataFrame([[BA, MCC, SN, SP, PR]],
                    columns = ['BA', 'MCC', 'SN', 'SP', 'PR'])])

    results_df = results_df.query('MCC != 0')
    print('BA: '+str(results_df.BA.mean())+ ' +/- ' +str(results_df.BA.std()))
    print('MCC: '+str(results_df.MCC.mean())+ ' +/- ' +str(results_df.MCC.std()))
    print('SN: '+str(results_df.SN.mean())+ ' +/- ' +str(results_df.SN.std()))
    print('SP: '+str(results_df.SP.mean())+ ' +/- ' +str(results_df.SP.std()))

### known chemistry case

In [ ]:
x = smiles_df
y= smiles_df.Tox60

model = KNeighborsClassifier(n_neighbors=1, weights = 'uniform', metric = 'jaccard', n_jobs = -1)

veryAcuteClassifier (x, y, model, nb_cross_cross_validation = 10, nb_fold = 10, SEED = 24, case_chemistry = 'known')

### new chemistry case

In [ ]:
x = smiles_df
y= smiles_df.Tox60
butinaCluster = smiles_df.cluster

model = KNeighborsClassifier(n_neighbors=1, weights = 'uniform', metric = 'jaccard', n_jobs = -1)

veryAcuteClassifier (x, y, model, nb_cross_cross_validation = 10, nb_fold = 10, SEED = 24, case_chemistry = 'new', clusters = butinaCluster)

# morphological profiles based classifier

In [ ]:
all_singleMorphologicalProfiles_df

## feature selection

In [ ]:
# performed at single profile level (not provided)
features = all_singleMorphologicalProfiles_df.loc[:, feature_cols].columns.tolist()
metadata = all_singleMorphologicalProfiles_df.drop(feature_cols, axis="columns").columns.tolist()
feature_select_opts = [
    "blocklist",
    "drop_outliers",
    "variance_threshold",
    "correlation_threshold",
    "noise_removal",
]

all_data_cleaned = pycytominer.feature_select(
    profiles=all_singleMorphologicalProfiles_df, 
    features=features,
    samples="all",
    operation=feature_select_opts,
    corr_method="pearson",
    outlier_cutoff=500,
    freq_cut=0.05,
    unique_cut=0.01,
    corr_threshold=0.9,
    noise_removal_perturb_groups = 'Metadata_BCS_conc',
    noise_removal_stdev_cutoff = 1.2,
)

## classifiers

### known chemistry case

In [8]:
x = cellPainting_consensusMorphologicalProfile_acuteToxicity.query('Metadata_concentration == 31.6').loc[:, CaravaggioDalitFeatSeat01]
y= cellPainting_consensusMorphologicalProfile_acuteToxicity.query('Metadata_concentration == 31.6').loc[:, 'Tox60']

model = KNeighborsClassifier(n_neighbors=1, weights = 'uniform', metric = 'correlation', n_jobs = -1)

veryAcuteClassifier (x, y, model, nb_cross_cross_validation = 10, nb_fold = 10, SEED = 24, case_chemistry = 'known')

BA: 0.744452096724824 +/- 0.07887693787674932
MCC: 0.4995151528371864 +/- 0.15919366782765879
SN: 0.7033976124885217 +/- 0.12496304637503236
SP: 0.7855065809611265 +/- 0.11830071909464804


### new chemistry case

In [9]:
x = cellPainting_consensusMorphologicalProfile_acuteToxicity.query('Metadata_concentration == 31.6').loc[:, CaravaggioDalitFeatSeat01]
y= cellPainting_consensusMorphologicalProfile_acuteToxicity.query('Metadata_concentration == 31.6').loc[:, 'Tox60']
butinaCluster = cellPainting_consensusMorphologicalProfile_acuteToxicity.query('Metadata_concentration == 31.6').loc[:, 'cluster']

model = KNeighborsClassifier(n_neighbors=1, weights = 'uniform', metric = 'correlation', n_jobs = -1)

veryAcuteClassifier (x, y, model, nb_cross_cross_validation = 10, nb_fold = 10, SEED = 24, case_chemistry = 'new', clusters = butinaCluster)

BA: 0.7193065207832812 +/- 0.11883594111574444
MCC: 0.41765812443150213 +/- 0.22523895833792934
SN: 0.6633066363931305 +/- 0.21878613520945045
SP: 0.7753064051734322 +/- 0.14730494511774617


# Decision support model

In [ ]:
# dataframe with cell painting morphological profile for 31.6µM, with Morgan Fingerprint, with acute toxicity label, and with butina cluster
cellPainting_consensusMorphologicalProfile_MorganFP_acuteToxicity

In [142]:
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.svm import SVC
import random
class DecisionSupportModel(ClassifierMixin, BaseEstimator):

    def __init__(self, clf):
        # the two KNN classifiers
        self.KNN_CP31 = KNeighborsClassifier(n_neighbors=1, weights = 'uniform', metric = 'correlation', n_jobs = -1)
        self.KNN_MFP = KNeighborsClassifier(n_neighbors=1, weights = 'uniform', metric = 'jaccard', n_jobs = -1)
        # the decision support classifier
        self.clf = clf
        # the columns for the cell painting features
        self.CPfeatureSet = CaravaggioDalitFeatSeat01
        # the columns for the Morgan Fingerprint
        self.MFfeatureSet = [str(num) for num in range(1024)]

    def fit(self, X, y):
 
        self.X_ = X
        self.y_ = y
       
        # training of the two KNNs
        self.KNN_CP31 = self.KNN_CP31.fit(self.X_[self.CPfeatureSet], self.y_)
        self.KNN_MFP = self.KNN_MFP.fit(self.X_[self.MFfeatureSet], self.y_)
        
        # we get the first kneighbor distances for both KNN and we create a df
        KNN_res = pd.DataFrame([self.KNN_CP31.kneighbors()[0].ravel(),
        self.X_.iloc[self.KNN_CP31.kneighbors()[1].ravel()].Tox60.values,  
        self.KNN_MFP.kneighbors()[0].ravel(),
        self.X_.iloc[self.KNN_MFP.kneighbors()[1].ravel()].Tox60.values, y]).T
        KNN_res.columns = ['d_CP', 'CP_pred', 'd_MFP', 'MFP_pred', 'Tox60']
        # subset when predictions of the two KNN do not agree
        KNN_res = KNN_res.query('CP_pred != MFP_pred')

        # synthetic data
        KNN_res_more = KNN_res.copy()
        KNN_res_more = KNN_res_more.query('CP_pred == Tox60')
        random.seed(1)
        # we create synthetic data
        KNN_res_more.loc[:, 'd_MFP'] = [random.uniform(0.7, 0.9) for _ in range(KNN_res_more.shape [0])]
        # we add them to the training set
        KNN_res = pd.concat([KNN_res, KNN_res_more])
        
        # we fit the decision support classifier with the training set
        self.clf = self.clf.fit(KNN_res.iloc[:, :-1], KNN_res.iloc[:, -1])
        
        return self

    def predict(self, X):
  
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])

        # df with predictions and distances of the nearest neighbors for the two KNN
        KNN_res_test = pd.DataFrame([self.KNN_CP31.kneighbors(X[self.CPfeatureSet])[0].ravel(),
        self.KNN_CP31.predict(X[self.CPfeatureSet]),
        self.KNN_MFP.kneighbors(X[self.MFfeatureSet])[0].ravel(),
        self.KNN_MFP.predict(X[self.MFfeatureSet])]).T
        KNN_res_test.columns = ['d_CP', 'CP_pred', 'd_MFP', 'MFP_pred']
        
        # we create the prediction column which takes per default le Cell Painting KNN prediction
        KNN_res_test.loc[:, 'pred'] = KNN_res_test.loc[:, 'CP_pred'].values
        

        # case when the two KNN do not have the same predicions -> we use the decision support model
        if KNN_res_test.loc[KNN_res_test.CP_pred != KNN_res_test.MFP_pred, 'pred'].shape[0] > 0:
            KNN_res_test.loc[KNN_res_test.CP_pred != KNN_res_test.MFP_pred, 'pred'] = self.clf.predict(KNN_res_test.loc[KNN_res_test.CP_pred != KNN_res_test.MFP_pred, ['d_CP', 'CP_pred', 'd_MFP', 'MFP_pred']])
        # we return the predictions
        return KNN_res_test.loc[:, 'pred'].values

## known chemistry case

In [ ]:
x = cellPainting_consensusMorphologicalProfile_MorganFP_acuteToxicity
y= cellPainting_consensusMorphologicalProfile_MorganFP_acuteToxicity.Tox60

model = DecisionSupportModel((clf = SVC(C = 8, class_weight = 'balanced')))

veryAcuteClassifier (x, y, model, nb_cross_cross_validation = 10, nb_fold = 10, SEED = 24, case_chemistry = 'new', clusters = butinaCluster)

## new chemistry case

In [ ]:
x = cellPainting_consensusMorphologicalProfile_MorganFP_acuteToxicity
y= cellPainting_consensusMorphologicalProfile_MorganFP_acuteToxicity.Tox60
butinaCluster = cellPainting_consensusMorphologicalProfile_MorganFP_acuteToxicity.cluster

model = DecisionSupportModel((clf = SVC(C = 8, class_weight = 'balanced')))

veryAcuteClassifier (x, y, model, nb_cross_cross_validation = 10, nb_fold = 10, SEED = 24, case_chemistry = 'new', clusters = butinaCluster)

# statistical test

In [ ]:
# source https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_stats.html
import numpy as np
from scipy.stats import t


def corrected_std(differences, n_train, n_test):
    """Corrects standard deviation using Nadeau and Bengio's approach.

    Parameters
    ----------
    differences : ndarray of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    corrected_std : float
        Variance-corrected standard deviation of the set of differences.
    """
    # kr = k times r, r times repeated k-fold crossvalidation,
    # kr equals the number of times the model was evaluated
    kr = len(differences)
    corrected_var = np.var(differences, ddof=1) * (1 / kr + n_test / n_train)
    corrected_std = np.sqrt(corrected_var)
    return corrected_std


def compute_corrected_ttest(differences, df, n_train, n_test):
    """Computes right-tailed paired t-test with corrected variance.

    Parameters
    ----------
    differences : array-like of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    df : int
        Degrees of freedom.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    t_stat : float
        Variance-corrected t-statistic.
    p_val : float
        Variance-corrected p-value.
    """
    mean = np.mean(differences)
    std = corrected_std(differences, n_train, n_test)
    t_stat = mean / std
    p_val = t.sf(np.abs(t_stat), df)  # right-tailed t-test
    return t_stat, p_val

model_1_scores = model_scores.iloc[0].values  # scores of the best model
model_2_scores = model_scores.iloc[1].values  # scores of the second-best model

differences = model_1_scores - model_2_scores

n = differences.shape[0]  # number of test sets
df = n - 1
n_train = len(list(cv.split(X, y))[0][0])
n_test = len(list(cv.split(X, y))[0][1])

t_stat, p_val = compute_corrected_ttest(differences, df, n_train, n_test)
print(f"Corrected t-value: {t_stat:.3f}\nCorrected p-value: {p_val:.3f}")